<a href="https://colab.research.google.com/github/0Nguyen0Cong0Tuan0/Road-Buddy-Challenge/blob/main/models/yolo_finetune_both.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **YOLO11 Unified Dataset Training: Road Lane + BDD100K**

This notebook trains YOLO models on a **unified dataset** combining:
- **Road Lane v2** (6 lane type classes)
- **BDD100K** (11 traffic classes, excluding generic 'lane')

> **Note**: This approach prevents catastrophic forgetting by training on all 17 classes simultaneously.

**Unified Classes (17 total)**

| ID | Class | Source |
|----|-------|--------|
| 0-5 | Lane types (divider, dotted, double, random, road-sign, solid) | Road Lane |
| 6-16 | Traffic objects (bike, bus, car, drivable area, motor, person, rider, traffic light, traffic sign, train, truck) | BDD100K |

## **Setup & Installation**

In [ ]:
!pip install ultralytics -q

In [ ]:
import ultralytics
import os
import yaml
import time
from collections import Counter
import numpy as np
from pathlib import Path
from ultralytics import YOLO
import shutil
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import torch
import seaborn as sns

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {DEVICE}')
if DEVICE == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

## **Dataset Paths**

In [ ]:
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'models' else Path.cwd()
DATA_DIR = PROJECT_ROOT / 'drive' / 'MyDrive' / 'traffic datasets'
MODELS_DIR = PROJECT_ROOT / 'drive' / 'MyDrive' / 'models'
RUNS_DIR = PROJECT_ROOT / 'drive' / 'MyDrive' / 'runs'

# Unified dataset path
UNIFIED_DATASET = {
    'path': DATA_DIR / 'unified',
    'yaml': DATA_DIR / 'unified' / 'data_unified.yaml',
    'description': 'Unified Road Lane + BDD100K dataset (17 classes)',
    'classes': [
        # Road Lane classes (0-5)
        'divider-line', 'dotted-line', 'double-line',
        'random-line', 'road-sign-line', 'solid-line',
        # BDD100K classes (6-16)
        'bike', 'bus', 'car', 'drivable area', 'motor',
        'person', 'rider', 'traffic light', 'traffic sign', 'train', 'truck'
    ]
}

# Class grouping for analysis
LANE_CLASSES = list(range(0, 6))   # Classes 0-5
TRAFFIC_CLASSES = list(range(6, 17))  # Classes 6-16

# Verify dataset
if UNIFIED_DATASET['yaml'].exists():
    print(f"Unified dataset found: {UNIFIED_DATASET['path']}")
    with open(UNIFIED_DATASET['yaml'], 'r') as f:
        config = yaml.safe_load(f)
        print(f"  Classes: {config['nc']}")
        print(f"  Lane types: {len(LANE_CLASSES)} classes")
        print(f"  Traffic objects: {len(TRAFFIC_CLASSES)} classes")
else:
    print(f"Unified dataset NOT found at {UNIFIED_DATASET['yaml']}")
    print("Please run the merge_datasets.py script first!")

## **Enhanced Training Configuration**

Optimized training settings with:
- **100 epochs** for better convergence
- **Cosine learning rate scheduler** for smooth decay
- **Warmup epochs** to stabilize early training
- **Data augmentation** for robustness

In [ ]:
# Enhanced training configuration
TRAINING_CONFIG = {
    # Core settings
    'epochs': 5,             # More epochs for better convergence
    'imgsz': 640,            # Image size
    'batch': 16,             # Batch size (reduce if GPU OOM)
    'patience': 20,          # Early stopping patience
    'device': DEVICE,        # Training device
    'workers': 4,            # Data loader workers
    
    # Learning rate settings
    'lr0': 0.01,             # Initial learning rate
    'lrf': 0.01,             # Final learning rate (lr0 * lrf)
    'warmup_epochs': 3,      # Warmup epochs for stable start
    'warmup_momentum': 0.8,  # Warmup initial momentum
    'warmup_bias_lr': 0.1,   # Warmup initial bias lr
    'cos_lr': True,          # Use cosine learning rate scheduler
    
    # Optimizer settings
    'optimizer': 'AdamW',    # AdamW optimizer (better than SGD for fine-tuning)
    'momentum': 0.937,       # SGD momentum/Adam beta1
    'weight_decay': 0.0005,  # Optimizer weight decay
    
    # Data augmentation
    'hsv_h': 0.015,          # HSV-Hue augmentation
    'hsv_s': 0.7,            # HSV-Saturation augmentation
    'hsv_v': 0.4,            # HSV-Value augmentation
    'degrees': 0.0,          # Rotation augmentation (disabled for driving scenes)
    'translate': 0.1,        # Translation augmentation
    'scale': 0.5,            # Scale augmentation
    'shear': 0.0,            # Shear augmentation (disabled)
    'flipud': 0.0,           # Vertical flip (disabled - unnatural for driving)
    'fliplr': 0.5,           # Horizontal flip
    'mosaic': 1.0,           # Mosaic augmentation
    'mixup': 0.1,            # Mixup augmentation
    
    # Output settings
    'save': True,            # Save checkpoints
    'save_period': 10,       # Save checkpoint every N epochs
    'plots': True,           # Generate training plots
    'verbose': True,         # Verbose output
}

print("\nCore Settings:")
for key in ['epochs', 'imgsz', 'batch', 'patience', 'device']:
    print(f"   {key}: {TRAINING_CONFIG[key]}")

print("\nLearning Rate:")
for key in ['lr0', 'lrf', 'warmup_epochs', 'cos_lr', 'optimizer']:
    print(f"   {key}: {TRAINING_CONFIG[key]}")

print("\nData Augmentation:")
for key in ['mosaic', 'mixup', 'fliplr', 'scale', 'translate']:
    print(f"   {key}: {TRAINING_CONFIG[key]}")

## **Train YOLO11n on Unified Dataset**

Training from pretrained YOLO11n model on the unified dataset with 17 classes.
This ensures the model can detect both lane types AND traffic objects.

In [ ]:
# Load fresh pretrained YOLO11n (NOT Road Lane fine-tuned)
print('Loading pretrained YOLO11n...')
model_n = YOLO('yolo11n.pt')
model_n.info()

In [ ]:
# Train YOLO11n on unified dataset
print('Training YOLO11n on Unified Dataset (17 classes)')
print('This model will detect BOTH lane types AND traffic objects!')
print(f'\nUsing cosine LR scheduler: {TRAINING_CONFIG["lr0"]} -> {TRAINING_CONFIG["lr0"] * TRAINING_CONFIG["lrf"]}')

start_time = time.time()

results_n = model_n.train(
    data=str(UNIFIED_DATASET['yaml']),
    project=str(RUNS_DIR / 'finetune'),
    name='yolo11n_unified',
    exist_ok=True,
    **TRAINING_CONFIG
)

training_time_n = time.time() - start_time
print(f'\n YOLO11n training completed in {training_time_n/3600:.2f} hours')

## **Train YOLO11l on Unified Dataset**

In [ ]:
# Load fresh pretrained YOLO11l (NOT Road Lane fine-tuned)
print('Loading pretrained YOLO11l...')
model_l = YOLO('yolo11l.pt')
model_l.info()

In [ ]:
# Train YOLO11l on unified dataset (reduced batch size for larger model)
print('Training YOLO11l on Unified Dataset (17 classes)')

config_l = TRAINING_CONFIG.copy()
config_l['batch'] = 8  # Reduced batch size for larger model

start_time = time.time()

results_l = model_l.train(
    data=str(UNIFIED_DATASET['yaml']),
    project=str(RUNS_DIR / 'finetune'),
    name='yolo11l_unified',
    exist_ok=True,
    **config_l
)

training_time_l = time.time() - start_time
print(f'\n YOLO11l training completed in {training_time_l/3600:.2f} hours')

## **Comprehensive Validation**

Detailed validation metrics including:
- **Overall metrics**: mAP50, mAP50-95, Precision, Recall, F1
- **Per-class performance**: Separate analysis for lane types vs traffic objects
- **Confusion matrix**: Visualize misclassifications
- **Performance comparison**: YOLO11n vs YOLO11l

In [ ]:
def load_trained_model(runs_dir, model_name):
    """Load a trained model from the runs directory."""
    model_path = runs_dir / 'finetune' / model_name / 'weights' / 'best.pt'
    if model_path.exists():
        return YOLO(str(model_path))
    else:
        print(f"Model not found: {model_path}")
        return None

# Load trained models
print('Loading trained models...')
model_n_trained = load_trained_model(RUNS_DIR, 'yolo11n_unified')
model_l_trained = load_trained_model(RUNS_DIR, 'yolo11l_unified')

In [ ]:
def evaluate_model(model, dataset_yaml, model_name, class_names):
    """Comprehensive model evaluation."""
    print(f"Evaluating {model_name}")
    
    # Run validation
    metrics = model.val(
        data=str(dataset_yaml),
        split='test',
        verbose=False
    )
    
    # Overall metrics
    print("\nOverall Metrics:")
    print(f"   mAP50:      {metrics.box.map50:.4f}")
    print(f"   mAP50-95:   {metrics.box.map:.4f}")
    print(f"   Precision:  {metrics.box.mp:.4f}")
    print(f"   Recall:     {metrics.box.mr:.4f}")
    f1 = 2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr + 1e-6)
    print(f"   F1 Score:   {f1:.4f}")
    
    # Per-class AP50
    ap50_per_class = metrics.box.ap50
    
    # Lane classes performance
    print("\nLane Types Performance:")
    lane_aps = []
    for i in LANE_CLASSES:
        if i < len(ap50_per_class):
            ap = ap50_per_class[i]
            lane_aps.append(ap)
            print(f"   {class_names[i]}: {ap:.4f}")
    print(f"   Average Lane mAP50: {np.mean(lane_aps):.4f}")
    
    # Traffic classes performance
    print("\nTraffic Objects Performance:")
    traffic_aps = []
    for i in TRAFFIC_CLASSES:
        if i < len(ap50_per_class):
            ap = ap50_per_class[i]
            traffic_aps.append(ap)
            print(f"   {class_names[i]}: {ap:.4f}")
    print(f"   Average Traffic mAP50: {np.mean(traffic_aps):.4f}")
    
    return {
        'model_name': model_name,
        'map50': metrics.box.map50,
        'map': metrics.box.map,
        'precision': metrics.box.mp,
        'recall': metrics.box.mr,
        'f1': f1,
        'lane_map50': np.mean(lane_aps),
        'traffic_map50': np.mean(traffic_aps),
        'ap50_per_class': ap50_per_class
    }

In [ ]:
# Evaluate both models
class_names = UNIFIED_DATASET['classes']
results_dict = {}

if model_n_trained:
    results_dict['yolo11n'] = evaluate_model(
        model_n_trained, 
        UNIFIED_DATASET['yaml'], 
        'YOLO11n Unified',
        class_names
    )

if model_l_trained:
    results_dict['yolo11l'] = evaluate_model(
        model_l_trained, 
        UNIFIED_DATASET['yaml'], 
        'YOLO11l Unified',
        class_names
    )

In [ ]:
# Model comparison visualization
if len(results_dict) >= 2:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Overall metrics comparison
    metrics_to_compare = ['map50', 'map', 'precision', 'recall', 'f1']
    x = np.arange(len(metrics_to_compare))
    width = 0.35
    
    vals_n = [results_dict['yolo11n'][m] for m in metrics_to_compare]
    vals_l = [results_dict['yolo11l'][m] for m in metrics_to_compare]
    
    axes[0].bar(x - width/2, vals_n, width, label='YOLO11n', color='#3498db')
    axes[0].bar(x + width/2, vals_l, width, label='YOLO11l', color='#e74c3c')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(['mAP50', 'mAP50-95', 'Precision', 'Recall', 'F1'])
    axes[0].set_ylabel('Score')
    axes[0].set_title('Overall Metrics Comparison')
    axes[0].legend()
    axes[0].set_ylim(0, 1)
    
    # Lane vs Traffic performance
    categories = ['Lane Types', 'Traffic Objects']
    x = np.arange(len(categories))
    
    vals_n = [results_dict['yolo11n']['lane_map50'], results_dict['yolo11n']['traffic_map50']]
    vals_l = [results_dict['yolo11l']['lane_map50'], results_dict['yolo11l']['traffic_map50']]
    
    axes[1].bar(x - width/2, vals_n, width, label='YOLO11n', color='#3498db')
    axes[1].bar(x + width/2, vals_l, width, label='YOLO11l', color='#e74c3c')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(categories)
    axes[1].set_ylabel('mAP50')
    axes[1].set_title('Lane vs Traffic Detection')
    axes[1].legend()
    axes[1].set_ylim(0, 1)
    
    # Per-class AP50 heatmap
    ap_data = np.array([
        results_dict['yolo11n']['ap50_per_class'],
        results_dict['yolo11l']['ap50_per_class']
    ])
    
    im = axes[2].imshow(ap_data, aspect='auto', cmap='RdYlGn', vmin=0, vmax=1)
    axes[2].set_yticks([0, 1])
    axes[2].set_yticklabels(['YOLO11n', 'YOLO11l'])
    axes[2].set_xticks(range(len(class_names)))
    axes[2].set_xticklabels(class_names, rotation=45, ha='right', fontsize=8)
    axes[2].set_title('Per-Class AP50')
    plt.colorbar(im, ax=axes[2], label='AP50')
    
    plt.tight_layout()
    plt.savefig(str(RUNS_DIR / 'finetune' / 'unified_model_comparison.png'), dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n Comparison plot saved to: {RUNS_DIR / 'finetune' / 'unified_model_comparison.png'}")

## **Results Summary**

In [ ]:
# Create summary table
if results_dict:
    summary_data = []
    for name, res in results_dict.items():
        summary_data.append({
            'Model': res['model_name'],
            'mAP50': f"{res['map50']:.4f}",
            'mAP50-95': f"{res['map']:.4f}",
            'Precision': f"{res['precision']:.4f}",
            'Recall': f"{res['recall']:.4f}",
            'F1': f"{res['f1']:.4f}",
            'Lane mAP50': f"{res['lane_map50']:.4f}",
            'Traffic mAP50': f"{res['traffic_map50']:.4f}"
        })
    
    summary_df = pd.DataFrame(summary_data)
    display(summary_df)
    
    # Save to CSV
    summary_df.to_csv(str(RUNS_DIR / 'finetune' / 'unified_model_summary.csv'), index=False)
    print(f"\nSummary saved to: {RUNS_DIR / 'finetune' / 'unified_model_summary.csv'}")

## **Visual Inference Test**

Test the model on sample images to verify it detects both lane types and traffic objects.

In [ ]:
# Test inference on sample images from both sources
test_images_dir = UNIFIED_DATASET['path'] / 'test' / 'images'

if test_images_dir.exists() and model_n_trained:
    # Get samples from both Road Lane and BDD100K
    all_images = list(test_images_dir.glob('*.jpg')) + list(test_images_dir.glob('*.png'))
    
    # Try to get mix of both sources
    road_lane_images = [p for p in all_images if 'roadlane' in p.name.lower()][:3]
    bdd100k_images = [p for p in all_images if 'bdd100k' in p.name.lower()][:3]
    sample_images = road_lane_images + bdd100k_images
    
    if not sample_images:
        sample_images = all_images[:6]
    
    if sample_images:
        print(f"Running inference on {len(sample_images)} test images...\n")
        
        n_images = len(sample_images)
        cols = 3
        rows = (n_images + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
        axes = axes.flatten() if n_images > cols else [axes] if n_images == 1 else axes
        
        for idx, img_path in enumerate(sample_images):
            results = model_n_trained.predict(str(img_path), verbose=False, conf=0.25)
            
            # Get annotated image
            annotated = results[0].plot()
            annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
            
            axes[idx].imshow(annotated)
            
            # Determine source
            source = "Road Lane" if 'roadlane' in img_path.name.lower() else "BDD100K"
            axes[idx].set_title(f"[{source}] {img_path.name[:30]}...", fontsize=9)
            axes[idx].axis('off')
            
            # Count detections by category
            if results[0].boxes is not None and len(results[0].boxes) > 0:
                detected_classes = results[0].boxes.cls.cpu().numpy().astype(int)
                lane_count = sum(1 for c in detected_classes if c in LANE_CLASSES)
                traffic_count = sum(1 for c in detected_classes if c in TRAFFIC_CLASSES)
                print(f"{img_path.name}: Lanes: {lane_count}, Traffic: {traffic_count}")
            else:
                print(f"{img_path.name}: No detections")
        
        # Hide empty subplots
        for idx in range(len(sample_images), len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.savefig(str(RUNS_DIR / 'finetune' / 'unified_inference_samples.png'), dpi=150, bbox_inches='tight')
        plt.show()
    else:
        print("No test images found.")
else:
    print(f"Test directory not found or model not loaded.")